## Exploration of explicitly encoding Lexical Patterns into Neural Models

In [1]:
%ls

Lexical Exploration.ipynb  composition_learning.py
README.md                  file_util.py
__pycache__/               repeval_exploration.ipynb


In [2]:
%ls ..

GoogleNews-vectors-negative300.bin  notes.txt
amod.train.filtered                 repeval2017/


In [3]:
import numpy as np
import composition_learning
from gensim.models import KeyedVectors as embedding_model

Using Theano backend.


Zunächst erstmal Zielnomen, Nomen, Adjektive aus tsv-File laden

In [4]:
import csv
path_to_adj_noun_file = "../amod.train.filtered"
gnoun_noun_adj_list = [] #a list of lists, each sublist containing a goal-noun, noun, adjective triple
goal_noun_list = [] #only the goal nouns, needed for functions from composition_learning.py.
with open(path_to_adj_noun_file, "r") as tsv_file:
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    for row in tsv_reader:
        gnoun_noun_adj_list.append(row)
        goal_noun_list.append(row[0].upper())

Danach Word Embeddings aus Modell laden:

In [5]:
path_to_word_embeddings = "../GoogleNews-vectors-negative300.bin"
vector_space = embedding_model.load_word2vec_format(path_to_word_embeddings, binary=True)

Daten in das richtige Format für das Keras-Modell bringen:

In [ ]:
training_data, training_labels, not_in_list = composition_learning.construct_data_and_labels(gnoun_noun_adj_list,
                                                                                             vector_space,
                                                                                             goal_noun_list,
                                                                                             verbosity = 0
                                                                                            )

Das Modell trainieren. tensor_model bezeichnet das modell, indem adjektiv mit Tensor zusammen zu einer Matrix multipliziert wird, die dann mit Nomen-vektor multipliziert wird um wieder einen Vektor zu erhalten.
weighted_model steht für ein additives Modell mit Gewichtigunsmatrizen für je Adjetiv und Nomen.

In [ ]:
tensor_model = composition_learning.train_model(training_data, training_labels, 
                                         composition_mode = 'weighted_adj_and_noun_add_identity', verbosity=1)

weighted_model = composition_learning.train_model(training_data, training_labels,
                                                composition_mode = 'tensor_mult_identity', verbosity=1)

Trainiere NN mit mode weighted_adj_and_noun_add_identity...
Trainiere NN mit mode tensor_mult_identity...


Um mit den Modellen Vektoren zu berechnen, tut man folgendes:

In [ ]:
goal_noun, noun, adj = gnoun_noun_adj_list[0]


goal_noun_tensor_vector = tensor_model.predict(np.asarray([[vectorspace[adj], vectorspace[noun]]]))[0, 0]
goal_noun_weighted_add_vector = weighted_model.predict(np.asarray([[vectorspace[adj], vectorspace[noun]]]))[0]

Todo:
+ Matthias Fehler nachvollziehen
+ Turney Datensatz laden, in ordentliche Struktur bringen
+ Evaluieren:
    - Modell je den besten kandidaten wählen lassen
         - wie viele sind korrekt gewählt? wie viele falsche sind korrekterweise nicht gewählt? (nachlesen)
         - average rank des korrekten Kandidaten (nachlesen)